# Test bug on RAGxplorer on query
Without brackets around the query in the openai client with chroma, embeddings are created for each word, not each string.

In [ ]:
import os
from ragxplorer import RAGxplorer

from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv(),override=True)
import json
import chromadb
import pandas as pd

In [ ]:
query="What are examples of lubricants which should be avoided for space mechanism applications?"

OPENAI_API_KEY=os.getenv('OPENAI_API_KEY')
embedding_model='text-embedding-ada-002'

data_path='../data/AMS/'
pdf='AMS_2022.pdf'

index_path='../db/chromadb/'
index_name='chromadb-openai-ams'

viz_data='../data/AMS/ams_data-400-0-50.json'

In [ ]:
with open(viz_data, 'r') as f:
    data = json.load(f)

index_name=data['visualization_index_name']
umap_params=data['umap_params']
viz_data=pd.read_json(data['viz_data'], orient='split')

In [ ]:
rx_client = RAGxplorer(embedding_model=embedding_model)
chroma_client = chromadb.PersistentClient(path='../db'+'/chromadb/')
collection=chroma_client.get_collection(name=index_name,embedding_function=rx_client._chosen_embedding_model)

In [ ]:
rx_client._query.original_query = query

In [ ]:
rx_client.load_chroma(collection,
                     umap_params=umap_params,
                     initialize_projector=True)

In [ ]:
# All of these do the same thing, tested

query_embedding_chroma=rx_client._chosen_embedding_model([rx_client._query.original_query])
# query_embedding=rx_client._vectordb._embedding_function(rx_client._query.original_query)
# query_embedding=embedding_function(query)

In [ ]:
len(rx_client._query.original_query)

In [ ]:
fig = rx_client.visualize_query(query,
                                import_projection_data=viz_data,
                                verbose=True)

In [ ]:
from openai import OpenAI
client = OpenAI()

# Done correctly, see here: https://platform.openai.com/docs/guides/embeddings/use-cases
openai_embedding=client.embeddings.create(input = [query], model=embedding_model).data[0].embedding

# Endpoints

In [ ]:
import json

config_file = '../config/config.json'
with open(config_file, 'r') as f:
        config = json.load(f)
        databases = {db['name']: db for db in config['databases']}
        llms  = {m['name']: m for m in config['llms']}

In [ ]:
hf_models=llms['Hugging Face']['models']
model_names= [item['model'] for item in hf_models]
model_endpoints= [item['endpoint'] for item in hf_models]


In [ ]:
model_name = "NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO"
endpoint = next(item['endpoint'] for item in hf_models if item['model'] == model_name)

In [ ]:
from langchain_openai import OpenAI, ChatOpenAI
# from langchain_community.llms import HuggingFaceTextGenInference
from langchain_community.llms import HuggingFaceHub
import os

from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv(),override=True)

api_key=os.getenv('HUGGINGFACEHUB_API_TOKEN')

# llm = ChatOpenAI(
#                     model_name="tgi",
#                     openai_api_key=api_key,
#                     openai_api_base=endpoint + "/v1/",
#                 )




# llm = HuggingFaceTextGenInference(
#     inference_server_url=endpoint,
#     # max_new_tokens=512,
#     # top_k=10,
#     # top_p=0.95,
#     # typical_p=0.95,
#     temperature=0.01,
#     # repetition_penalty=1.03,
# )
# llm("What did foo say about bar?")

# model_name = "NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO"
# model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
# model_name = "meta-llama/Llama-2-70b-chat-hf"

llm = HuggingFaceHub(repo_id=model_name,
                     model_kwargs={"temperature": 0.1, "max_length": 516})


In [ ]:
prompt="""
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.
---
Your name is Aerospace Chatbot. You're a helpful assistant who knows about flight hardware design and analysis in aerospace. If you don't know the answer, just say that you don't know, don't try to make up an answer.
Include sources from the chat history in the standalone question created.
---

Chat History:

User Question: What are examples of lubricants which should be avoided for space mechanism applications?
Standalone Question:
"""

response=llm.predict(prompt)
print(response)

# Testing

In [ ]:
import unittest
import os
import sys
import logging
import json
import itertools
from dotenv import load_dotenv,find_dotenv

from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import VoyageEmbeddings
from langchain_openai import ChatOpenAI

# Assuming `/home/user/projects/my_modules` is the path to the folder with your modules
sys.path.append('../src')

load_dotenv(find_dotenv(),override=True)



OPENAI_API_KEY=os.getenv('OPENAI_API_KEY')
VOYAGE_API_KEY=os.getenv('VOYAGE_API_KEY')
HUGGINGFACEHUB_API_TOKEN=os.getenv('HUGGINGFACEHUB_API_TOKEN')
PINECONE_API_KEY=os.getenv('PINECONE_API_KEY')
LOCAL_DB_PATH='.'   # Default to the test path for easy cleanup.

In [ ]:
llm={'OpenAI':ChatOpenAI(model_name='gpt-3.5-turbo-1106', # Openai
                                openai_api_key=OPENAI_API_KEY,
                                max_tokens=500), 
            'Hugging Face':ChatOpenAI(base_url='https://api-inference.huggingface.co/v1',  # Hugging face
                                    model='mistralai/Mistral-7B-Instruct-v0.2',
                                    api_key=HUGGINGFACEHUB_API_TOKEN,
                                    max_tokens=500)}
query_model={'OpenAI':OpenAIEmbeddings(model='text-embedding-ada-002',openai_api_key=OPENAI_API_KEY),
                    'Voyage':VoyageEmbeddings(voyage_api_key=VOYAGE_API_KEY),
                    'RAGatouille':'colbert-ir/colbertv2.0'}
index_type = {index: index for index in ['ChromaDB', 'Pinecone', 'RAGatouille']}
rag_type = {rag: rag for rag in ['Standard','Parent-Child','Summary']}

def permute_tests(test_data):
    """
    Permute test data to generate all possible combinations.
    Data is in a list of dicts, where each has keys to be iterated. 
    Example: [{'index_type': ['ChromaDB'], 'rag_type': ['Standard', 'Parent-Child', 'Summary']}, 
              {'index_type': [Pinecone], 'rag_type': ['Standard', 'Parent-Child']}]
    """
    rows = []
    for row_data in test_data:
        keys = row_data.keys()
        values = row_data.values()
        permutations = list(itertools.product(*values))
        for perm in permutations:
            row = dict(zip(keys, perm))
            rows.append(row)
    return rows

In [ ]:
# Determine the set of cases to screen
test_cases=[{
    # Tests ChromaDB setups, advanced RAG (standard/parent-child)
    'index_type': [index_type['ChromaDB']],
    'query_model': [query_model['OpenAI']],
    'rag_type': [rag_type['Standard'],rag_type['Parent-Child']],
    'llm': [llm['Hugging Face']]
},
{
    # Tests advanced RAG (summary) and LLM (openai/hugging face)
    'index_type': [index_type['ChromaDB']],
    'query_model': [query_model['OpenAI']],
    'rag_type': [rag_type['Summary']],
    'llm': [llm['OpenAI'],llm['Hugging Face']]
},
{
    # Tests Pinecone setups, embedding types (openai/voyage)
    'index_type': [index_type['Pinecone']],
    'query_model': [query_model['OpenAI'],query_model['Voyage']],
    'rag_type': [rag_type['Standard']],
    'llm': [llm['Hugging Face']]
},
{
    # Tests RAGatouille setup
    'index_type': [index_type['RAGatouille']],
    'query_model': [query_model['RAGatouille']],
    'rag_type': [rag_type['Standard']],
    'llm': [llm['Hugging Face']]
}]
tests=permute_tests(test_cases)

# Export the output with keys
keys = test_cases[0].keys()
output = [{**test, 'id': i+1, 'keys': keys} for i, test in enumerate(tests)]

In [ ]:
a=output[0]['llm']
type(a)